In [7]:
# --- Cell 1: setup (imports + parameters) ---

from pathlib import Path
from datetime import datetime, timezone, timedelta
from typing import Optional, Tuple
import re
import os

import numpy as np
import pandas as pd
import netCDF4 as netcdf
from netCDF4 import num2date

# ============== EDIT THESE IF NEEDED ==============
SITE        = "Lucinda"
TARGET_LAT  = -18.5198          # site latitude
TARGET_LON  = 146.386         # site longitude
HOURS_TOL   = 3                  # ± hours for the in-situ time-window match

# Base folder that contains your .nc files and CSVs
BASE_DIR    = Path("Y:/") / "Mingyue" / SITE

# Folder with many *.nc (SeaDAS L2) files
MAIN_DIR    = BASE_DIR / "l2gen_product"

# Output summary CSV from Cell 2
SUMMARY_CSV = BASE_DIR / f"{SITE}_l2_summary.csv"

# In-situ CSV (must contain columns like Date, Time, Chlorophyll-a)
INSITU_CSV  = BASE_DIR / f"{SITE}_in_situ_data.csv"

# Pattern to find NetCDF files
NC_GLOB_PATTERN = "*.nc"

# Whether to recurse into subfolders of MAIN_DIR
RECURSIVE_SEARCH = True
# ==================================================

# ---- Known aliases ----
LAT_ALIASES    = ['latitude', 'lat', 'Latitude']
LON_ALIASES    = ['longitude', 'lon', 'Longitude']
CHLOR_A_ALIASES= ['chlor_a', 'CHLORA', 'chlorophyll', 'chl_ocx', 'chl']
FLAGS_ALIASES  = ['l2_flags', 'l2flags', 'flags']

# ---- Common SeaDAS l2_flags bits (adjust if your product differs) ----
L2_FLAG_BITS = {
    0: "ATMFAIL", 1: "LAND", 2: "PRODWARN", 3: "HIGLINT", 4: "HILT",
    5: "HISATZEN", 6: "COASTZ", 7: "SPARE1", 8: "STRAYLIGHT", 9: "CLDICE",
    10: "COCCOLITH", 11: "TURBIDW", 12: "HISOLZEN", 13: "SPARE2", 14: "LOWLW",
    15: "CHLFAIL", 16: "NAVWARN", 17: "ABSAER", 18: "SPARE3", 19: "TRICHO",
    20: "SPARE4", 21: "MAXAERITER", 22: "MODGLINT", 23: "CHLWARN", 24: "ATMWARN",
    25: "SPARE5", 26: "SEAICE", 27: "NAVFAIL", 28: "FILTER", 29: "SPARE6",
    30: "SPARE7", 31: "HIPOL"
}

print(f"Using BASE_DIR={BASE_DIR}")
print(f"MAIN_DIR={MAIN_DIR}")
print(f"SUMMARY_CSV={SUMMARY_CSV}")
print(f"INSITU_CSV={INSITU_CSV}")


Using BASE_DIR=Y:\Mingyue\Lucinda
MAIN_DIR=Y:\Mingyue\Lucinda\l2gen_product
SUMMARY_CSV=Y:\Mingyue\Lucinda\Lucinda_l2_summary.csv
INSITU_CSV=Y:\Mingyue\Lucinda\Lucinda_in_situ_data.csv


In [8]:
# --- Cell 2: batch extract chlor_a + window stats + UTC acquisition datetime into SUMMARY_CSV ---

# ---------------- Utility helpers ----------------

def find_variable(data_source, aliases):
    """Search a dataset or group for the first variable matching any alias."""
    for alias in aliases:
        var = data_source.variables.get(alias)
        if var is not None:
            print(f"Found variable using alias: '{alias}'")
            return var
    return None

def get_var_anywhere(dataset, aliases,
                     group_candidates=('navigation_data', 'geophysical_data',
                                       'geolocation_data', 'nav', 'navigation', 'geophysical')):
    """Try to find a variable in common groups first, then at root."""
    for gname in group_candidates:
        grp = dataset.groups.get(gname)
        if grp:
            v = find_variable(grp, aliases)
            if v is not None:
                return v
    return find_variable(dataset, aliases)

def ensure_2d_latlon(lats, lons):
    """Support both 1D and 2D navigation arrays."""
    if lats.ndim == 1 and lons.ndim == 1:
        return np.meshgrid(lats, lons, indexing='ij')
    return lats, lons

def window_slice(center_row, center_col, half_size, nrows, ncols):
    r0 = max(0, center_row - half_size)
    r1 = min(nrows, center_row + half_size + 1)
    c0 = max(0, center_col - half_size)
    c1 = min(ncols, center_col + half_size + 1)
    return r0, r1, c0, c1

def window_stats(masked_array, r0, r1, c0, c1):
    box = masked_array[r0:r1, c0:c1]
    valid_count = int(np.ma.count(box))
    total = int((r1 - r0) * (c1 - c0))
    if valid_count > 0:
        return {
            "mean": float(np.ma.mean(box)),
            "median": float(np.ma.median(box)),
            "valid_pixels": valid_count,
            "total_pixels": total
        }
    else:
        return {"mean": None, "median": None, "valid_pixels": 0, "total_pixels": total}

def decode_flags(flag_val):
    if flag_val is None or np.ma.is_masked(flag_val):
        return []
    names = []
    ival = int(flag_val)
    for bit, name in L2_FLAG_BITS.items():
        if (ival >> bit) & 1:
            names.append(name)
    return names

def nearest_unmasked(chl, start_row, start_col, max_radius=50):
    """Find nearest unmasked, finite chlor_a pixel starting at (row, col)."""
    nrows, ncols = chl.shape
    if not np.ma.is_masked(chl[start_row, start_col]) and np.isfinite(chl[start_row, start_col]):
        return start_row, start_col, float(chl[start_row, start_col]), 0
    for rad in range(1, max_radius + 1):
        r0, r1, c0, c1 = window_slice(start_row, start_col, rad, nrows, ncols)
        window = chl[r0:r1, c0:c1]
        if np.ma.count(window) == 0:
            continue
        rr, cc = np.mgrid[r0:r1, c0:c1]
        dist2 = (rr - start_row) ** 2 + (cc - start_col) ** 2
        valid_mask = (~np.ma.getmaskarray(window)) & np.isfinite(window.filled(np.nan))
        if not np.any(valid_mask):
            continue
        dist2_valid = np.where(valid_mask, dist2, np.inf)
        flat_idx = np.argmin(dist2_valid)
        wr, wc = np.unravel_index(flat_idx, window.shape)
        rr_abs, cc_abs = r0 + wr, c0 + wc
        val = float(chl[rr_abs, cc_abs])
        d = int(np.sqrt(dist2[wr, wc]))
        return rr_abs, cc_abs, val, d
    return None, None, None, None

# ---------------- Rrs helpers ----------------
_RRS_RE = re.compile(r'^Rrs_(\d+)$', flags=re.IGNORECASE)

def find_rrs_bands(dataset):
    candidates = []
    all_containers = [dataset] + [dataset.groups[g] for g in dataset.groups]
    for container in all_containers:
        for name, var in container.variables.items():
            m = _RRS_RE.match(name)
            if m:
                wl = int(m.group(1))
                candidates.append((wl, var))
    candidates.sort(key=lambda x: x[0])
    return candidates

def read_rrs_at(rrs_vars, row, col):
    spectrum = {}
    for wl, var in rrs_vars:
        val = var[row, col]
        spectrum[wl] = None if np.ma.is_masked(val) or not np.isfinite(val) else float(val)
    return spectrum

# ---------------- Datetime extraction helpers ----------------
_FILENAME_DT_RE = re.compile(r'(\d{8}T\d{6})')

def _parse_iso_like(s: str) -> Optional[datetime]:
    if not s:
        return None
    s = s.strip()
    if s.endswith("Z"):
        s = s[:-1] + "+00:00"
    try:
        dt = datetime.fromisoformat(s)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

def _extract_datetime_from_attrs(ds) -> Optional[datetime]:
    attr_candidates = [
        "time_coverage_start", "time_coverage_center",
        "start_time", "StartTime", "start_datetime",
        "start_date", "product_start_time", "isotime", "isodate"
    ]
    for k in attr_candidates:
        if k in ds.__dict__:
            dt = _parse_iso_like(str(getattr(ds, k)))
            if dt:
                return dt
    # DOY-based triplet: start_year + start_day (+ start_msec or start_sec)
    if all(hasattr(ds, x) for x in ("start_year", "start_day")):
        try:
            year = int(getattr(ds, "start_year"))
            doy  = int(getattr(ds, "start_day"))
            base = datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(days=doy - 1)
            if hasattr(ds, "start_msec"):
                msec = float(getattr(ds, "start_msec"))
                base = base + timedelta(milliseconds=msec)
            elif hasattr(ds, "start_sec"):
                sec = float(getattr(ds, "start_sec"))
                base = base + timedelta(seconds=sec)
            return base
        except Exception:
            pass
    return None

def _extract_datetime_from_timevar(ds) -> Optional[datetime]:
    if "time" not in ds.variables:
        return None
    tv = ds.variables["time"]
    try:
        t0 = tv[()] if tv.shape == () else tv[0]
        units = getattr(tv, "units", None)
        calendar = getattr(tv, "calendar", "standard")
        if units:
            dt = num2date(t0, units=units, calendar=calendar)
            if isinstance(dt, np.ndarray):
                dt = dt.item()
            if getattr(dt, "tzinfo", None) is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.astimezone(timezone.utc)
    except Exception:
        return None
    return None

def _extract_datetime_from_filename(nc_path: Path) -> Optional[datetime]:
    m = _FILENAME_DT_RE.search(nc_path.name)
    if not m:
        return None
    token = m.group(1)  # YYYYMMDDTHHMMSS
    try:
        return datetime.strptime(token, "%Y%m%dT%H%M%S").replace(tzinfo=timezone.utc)
    except Exception:
        return None

def extract_acq_datetime_utc(nc_path: Path) -> Optional[datetime]:
    """Best-effort extraction of acquisition datetime (UTC)."""
    try:
        with netcdf.Dataset(str(nc_path), "r") as ds:
            dt = _extract_datetime_from_attrs(ds)
            if dt:
                return dt
            dt = _extract_datetime_from_timevar(ds)
            if dt:
                return dt
    except Exception:
        pass
    return _extract_datetime_from_filename(nc_path)

# ---------------- Row flattener & neighborhood flags ----------------
def _collect_neighborhood_flags(flags_var, row, col, compute_windows, nrows, ncols):
    neighborhood_flags = {}
    if flags_var is None:
        return {f"{w}x{w}": [] for w in compute_windows}
    flags = flags_var[:]
    for w in compute_windows:
        half = (w - 1) // 2
        r0, r1, c0, c1 = window_slice(row, col, half, nrows, ncols)
        if flags.ndim == 2:
            fwin = flags[r0:r1, c0:c1].astype(int)
            names = []
            for fv in np.unique(fwin):
                names.extend(decode_flags(fv))
            seen = set()
            uniq = [x for x in names if not (x in seen or seen.add(x))]
            neighborhood_flags[f"{w}x{w}"] = uniq
        else:
            neighborhood_flags[f"{w}x{w}"] = []
    return neighborhood_flags

def _safe_join_flags(names):
    if not names:
        return ""
    seen = set()
    uniq = [x for x in names if not (x in seen or seen.add(x))]
    return "|".join(uniq)

def _row_from_result(nc_path: Path, result: dict, acq_dt: Optional[datetime]) -> dict:
    win_keys = ["3x3", "5x5"]
    if acq_dt:
        acq_iso  = acq_dt.isoformat()
        acq_date = acq_dt.date().isoformat()
        acq_time = acq_dt.time().isoformat(timespec="seconds")
    else:
        acq_iso = acq_date = acq_time = ""

    row = {
        "file": str(nc_path),
        "filename": nc_path.name,
        "acq_datetime_utc": acq_iso,
        "acq_date_utc": acq_date,
        "acq_time_utc": acq_time,
        "nearest_row": result["nearest_pixel_index"]["row"],
        "nearest_col": result["nearest_pixel_index"]["col"],
        "nearest_lat": result["nearest_pixel_geo"]["lat"],
        "nearest_lon": result["nearest_pixel_geo"]["lon"],
        "center_chlor_a": result["center_value"],
        "center_flags": _safe_join_flags(result.get("center_flags", [])),
    }
    for wk in win_keys:
        ws = result["window_stats"].get(wk, {})
        row[f"{wk}_mean"]   = ws.get("mean")
        row[f"{wk}_median"] = ws.get("median")
        row[f"{wk}_valid"]  = ws.get("valid_pixels")
        row[f"{wk}_total"]  = ws.get("total_pixels")
        nfl = result.get("neighborhood_flags", {}).get(wk, [])
        row[f"{wk}_flags"] = _safe_join_flags(nfl)

    nv = result.get("nearest_valid_if_center_masked")
    if nv is not None:
        row["nearest_valid_value"]             = nv.get("value")
        row["nearest_valid_row"]               = nv.get("row")
        row["nearest_valid_col"]               = nv.get("col")
        row["nearest_valid_distance_pixels"]   = nv.get("distance_pixels")
        row["nearest_valid_flags"]             = _safe_join_flags(nv.get("flags", []))
    else:
        row["nearest_valid_value"]           = None
        row["nearest_valid_row"]             = None
        row["nearest_valid_col"]             = None
        row["nearest_valid_distance_pixels"] = None
        row["nearest_valid_flags"]           = ""
    return row

# ---------------- Main extractor ----------------
def get_chlor_a_at_location(
    l2_file_path: str,
    target_lat: float,
    target_lon: float,
    compute_windows=(3, 5),
    return_nearest_if_center_masked: bool = True
):
    if not os.path.exists(l2_file_path):
        raise FileNotFoundError(f"File not found: {l2_file_path}")

    with netcdf.Dataset(l2_file_path, 'r') as ds:
        lats_var  = get_var_anywhere(ds, LAT_ALIASES)
        lons_var  = get_var_anywhere(ds, LON_ALIASES)
        chl_var   = get_var_anywhere(ds, CHLOR_A_ALIASES)
        flags_var = get_var_anywhere(ds, FLAGS_ALIASES)

        if lats_var is None or lons_var is None:
            raise KeyError(f"Could not find latitude/longitude using {LAT_ALIASES} / {LON_ALIASES}")
        if chl_var is None:
            raise KeyError(f"Could not find chlorophyll-a using {CHLOR_A_ALIASES}")

        lats = lats_var[:]
        lons = lons_var[:]
        chl  = chl_var[:]

        lats, lons = ensure_2d_latlon(lats, lons)
        nrows, ncols = lats.shape

        lon_min, lon_max = np.nanmin(lons), np.nanmax(lons)
        t_lon = target_lon
        if (lon_max - lon_min) > 300:  # handle dateline
            candidates = [t_lon, t_lon - 360, t_lon + 360]
            diffs = [min(abs(c - lon_min), abs(c - lon_max)) for c in candidates]
            t_lon = candidates[int(np.argmin(diffs))]

        dist_sq = (lats - target_lat) ** 2 + (lons - t_lon) ** 2
        min_idx = np.argmin(dist_sq)
        row, col = np.unravel_index(min_idx, lats.shape)

        chl = np.ma.array(chl, copy=False)
        center_val = chl[row, col]
        center_val_out = None if np.ma.is_masked(center_val) or not np.isfinite(center_val) else float(center_val)

        center_flags = None
        if flags_var is not None:
            flags = flags_var[:]
            if flags.ndim == 2 and 0 <= row < flags.shape[0] and 0 <= col < flags.shape[1]:
                center_flags = int(flags[row, col])
            elif flags.ndim == 1:
                center_flags = int(min_idx)
        decoded_center_flags = decode_flags(center_flags) if center_flags is not None else []

        stats_by_window = {}
        for w in compute_windows:
            half = (w - 1) // 2
            r0, r1, c0, c1 = window_slice(row, col, half, nrows, ncols)
            stats_by_window[f"{w}x{w}"] = window_stats(chl, r0, r1, c0, c1)

        nearest_info = None
        nr = cc = nval = nd = None
        if center_val_out is None and return_nearest_if_center_masked:
            nr, cc, nval, nd = nearest_unmasked(chl, row, col, max_radius=50)
            if nr is not None:
                nearest_flags = None
                if flags_var is not None:
                    flags = flags_var[:]
                    if flags.ndim == 2:
                        nearest_flags = int(flags[nr, cc])
                    elif flags.ndim == 1:
                        nearest_flags = int(nr * ncols + cc)
                nearest_info = {
                    "row": int(nr),
                    "col": int(cc),
                    "value": float(nval),
                    "distance_pixels": int(nd),
                    "flags": decode_flags(nearest_flags) if nearest_flags is not None else []
                }

        rrs_vars = find_rrs_bands(ds)
        rrs_center  = read_rrs_at(rrs_vars, row, col) if rrs_vars else {}
        rrs_nearest = None
        if nr is not None and cc is not None and rrs_vars:
            rrs_nearest = read_rrs_at(rrs_vars, nr, cc)

        return {
            "target_input": {"lat": float(target_lat), "lon": float(target_lon)},
            "nearest_pixel_index": {"row": int(row), "col": int(col)},
            "nearest_pixel_geo": {"lat": float(lats[row, col]), "lon": float(lons[row, col])},
            "center_value": center_val_out,
            "window_stats": stats_by_window,
            "center_flags": decoded_center_flags,
            "neighborhood_flags": _collect_neighborhood_flags(flags_var, row, col, compute_windows, nrows, ncols),
            "nearest_valid_if_center_masked": nearest_info,
            "rrs_center": rrs_center,
            "rrs_nearest_valid": rrs_nearest
        }

def process_folder(
    main_dir: Path,
    target_lat: float,
    target_lon: float,
    out_csv: Path,
    pattern: str = "*.nc",
    compute_windows=(3, 5),
    return_nearest_if_center_masked=True,
    recursive=True
):
    main_dir = Path(main_dir)
    assert main_dir.exists(), f"Main dir not found: {main_dir}"

    paths = sorted(main_dir.rglob(pattern) if recursive else main_dir.glob(pattern))
    rows, errors = [], []

    print(f"[{datetime.now().isoformat(timespec='seconds')}] Found {len(paths)} NetCDF files under {main_dir}")
    for i, p in enumerate(paths, 1):
        try:
            acq_dt = extract_acq_datetime_utc(p)
            result = get_chlor_a_at_location(
                str(p),
                target_lat,
                target_lon,
                compute_windows=compute_windows,
                return_nearest_if_center_masked=return_nearest_if_center_masked
            )
            row = _row_from_result(p, result, acq_dt)
            rows.append(row)
            if i % 25 == 0:
                print(f"  processed {i}/{len(paths)}...")
        except Exception as e:
            errors.append((str(p), repr(e)))
            print(f"  ERROR on {p.name}: {e!r}")

    df = pd.DataFrame(rows)
    if not df.empty:
        df.sort_values(by=["acq_datetime_utc", "filename"], inplace=True, ignore_index=True)

    out_csv = Path(out_csv)
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_csv, index=False)
    print(f"[{datetime.now().isoformat(timespec='seconds')}] Wrote {len(df)} rows to {out_csv}")

    if errors:
        print("\nFiles with errors:")
        for f, msg in errors[:20]:
            print(f"  {f} -> {msg}")
        if len(errors) > 20:
            print(f"  ... and {len(errors)-20} more")

    return df, errors

# --------- Run the batch extraction ---------
df, errs = process_folder(
    main_dir=MAIN_DIR,
    target_lat=TARGET_LAT,
    target_lon=TARGET_LON,
    out_csv=SUMMARY_CSV,
    pattern=NC_GLOB_PATTERN,
    compute_windows=(3, 5),
    return_nearest_if_center_masked=True,
    recursive=RECURSIVE_SEARCH
)


[2025-10-23T16:12:25] Found 58 NetCDF files under Y:\Mingyue\Lucinda\l2gen_product
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using alias: 'l2_flags'
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using alias: 'l2_flags'
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using alias: 'l2_flags'
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using alias: 'l2_flags'
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using alias: 'l2_flags'
Found variable using alias: 'latitude'
Found variable using alias: 'longitude'
Found variable using alias: 'chlor_a'
Found variable using

In [ ]:
# # --- Cell 3: robust in-situ matchup (±HOURS_TOL hours) and write back to SUMMARY_CSV ---

# # Load satellite summary
# df_sat = pd.read_csv(SUMMARY_CSV)
# df_sat.columns = [c.strip() for c in df_sat.columns]

# # Parse acquisition datetime + fallback from separate date/time cols
# df_sat['acq_datetime_utc'] = pd.to_datetime(df_sat.get('acq_datetime_utc'), utc=True, errors='coerce')

# if 'acq_date_utc' in df_sat.columns and 'acq_time_utc' in df_sat.columns:
#     missing_mask = df_sat['acq_datetime_utc'].isna()
#     if missing_mask.any():
#         dt_fallback = pd.to_datetime(
#             df_sat.loc[missing_mask, 'acq_date_utc'].astype(str).str.strip() + " " +
#             df_sat.loc[missing_mask, 'acq_time_utc'].astype(str).str.strip(),
#             utc=True, errors='coerce'
#         )
#         df_sat.loc[missing_mask, 'acq_datetime_utc'] = dt_fallback

# # Load in-situ
# df_ins = pd.read_csv(INSITU_CSV)
# df_ins.columns = [c.strip() for c in df_ins.columns]

# # Detect column names
# date_col = next((c for c in df_ins.columns if c.lower() == 'date'), None)
# time_col = next((c for c in df_ins.columns if c.lower() == 'time'), None)
# chl_col  = next((c for c in df_ins.columns if c.lower() in {'chlorophyll-a','chlorophyll_a','chl_a','chl'}), None)

# if date_col is None or time_col is None or chl_col is None:
#     raise RuntimeError(
#         f"Could not find expected columns in {INSITU_CSV}. "
#         f"Have={list(df_ins.columns)}; need Date, Time, and Chlorophyll-a."
#     )

# # Build UTC timestamps (adjust here if in-situ is in local time)
# df_ins['insitu_datetime_utc'] = pd.to_datetime(
#     df_ins[date_col].astype(str).str.strip() + " " + df_ins[time_col].astype(str).str.strip(),
#     utc=True, errors='coerce'
# )
# df_ins['in-situ chl-a'] = pd.to_numeric(df_ins[chl_col], errors='coerce')
# df_ins = df_ins.dropna(subset=['insitu_datetime_utc']).copy()

# # Split satellite rows by timestamp availability
# sat_valid   = df_sat.dropna(subset=['acq_datetime_utc']).copy()
# sat_missing = df_sat[df_sat['acq_datetime_utc'].isna()].copy()

# # Sort (required for merge_asof)
# sat_valid  = sat_valid.sort_values('acq_datetime_utc').reset_index(drop=True)
# ins_sorted = df_ins.sort_values('insitu_datetime_utc').reset_index(drop=True)

# # Nearest-time join
# if not sat_valid.empty and not ins_sorted.empty:
#     merged = pd.merge_asof(
#         sat_valid,
#         ins_sorted[['insitu_datetime_utc', 'in-situ chl-a']].rename(columns={'insitu_datetime_utc':'match_time'}),
#         left_on='acq_datetime_utc',
#         right_on='match_time',
#         direction='nearest',
#         tolerance=pd.Timedelta(hours=HOURS_TOL)
#     ).drop(columns=['match_time'])
# else:
#     merged = sat_valid.copy()
#     merged['in-situ chl-a'] = np.nan

# # Unmatchable rows keep NaN
# if not sat_missing.empty:
#     sat_missing = sat_missing.copy()
#     sat_missing['in-situ chl-a'] = np.nan

# # Stitch + sort
# out = pd.concat([merged, sat_missing], ignore_index=True)
# sort_keys = [k for k in ['acq_datetime_utc','filename'] if k in out.columns]
# if sort_keys:
#     out = out.sort_values(sort_keys).reset_index(drop=True)

# # Save back to SUMMARY_CSV
# out.to_csv(SUMMARY_CSV, index=False)

# # Report
# n_total     = len(out)
# n_missing_t = sat_missing.shape[0]
# n_matched   = out['in-situ chl-a'].notna().sum()
# print(f"Matched in-situ chl-a within ±{HOURS_TOL} h for {n_matched}/{n_total} satellite rows.")
# print(f"Rows lacking satellite timestamps (could not be matched): {n_missing_t}")
# print(f"Updated file: {SUMMARY_CSV}")


In [9]:
#!/usr/bin/env python3
# -- coding: utf-8 --


# OUTPUT (will NOT overwrite inputs)

OUT_MATCHED_CSV = BASE_DIR / f"{SITE}_l2_summary_2.csv"

# Optional AERONET Rrs columns to carry through if present
AOC_RRS_COLS = [
    "aoc_rrs412","aoc_rrs443","aoc_rrs490","aoc_rrs510",
    "aoc_rrs560","aoc_rrs665","aoc_rrs681"
]
# ------------------------------------------------

def parse_satellite_times(df_sat: pd.DataFrame) -> pd.DataFrame:
    df_sat = df_sat.copy()
    df_sat.columns = [c.strip() for c in df_sat.columns]

    # Preferred combined datetime
    if "acq_datetime_utc" in df_sat.columns:
        df_sat["acq_datetime_utc"] = pd.to_datetime(
            df_sat["acq_datetime_utc"], utc=True, errors="coerce"
        )
    else:
        df_sat["acq_datetime_utc"] = pd.NaT

    # Fallback from separate date/time if needed
    if "acq_date_utc" in df_sat.columns and "acq_time_utc" in df_sat.columns:
        miss = df_sat["acq_datetime_utc"].isna()
        if miss.any():
            dt = pd.to_datetime(
                df_sat.loc[miss, "acq_date_utc"].astype(str).str.strip() + " " +
                df_sat.loc[miss, "acq_time_utc"].astype(str).str.strip(),
                utc=True, errors="coerce"
            )
            df_sat.loc[miss, "acq_datetime_utc"] = dt

    return df_sat

def load_insitu_aoc(df_ins: pd.DataFrame) -> pd.DataFrame:
    df_ins = df_ins.copy()
    df_ins.columns = [c.strip() for c in df_ins.columns]

    # Required AERONET-OC columns
    required = ["aoc_datetime", "aoc_chl_a"]
    for col in required:
        if col not in df_ins.columns:
            raise RuntimeError(f"In-situ file missing required column '{col}'")

    df_ins["aoc_datetime"] = pd.to_datetime(df_ins["aoc_datetime"], utc=True, errors="coerce")
    df_ins["aoc_chl_a"]    = pd.to_numeric(df_ins["aoc_chl_a"],    errors="coerce")
    df_ins = df_ins.dropna(subset=["aoc_datetime"]).reset_index(drop=True)

    keep_cols = ["aoc_site","aoc_latitude","aoc_longitude","aoc_datetime","aoc_chl_a"]
    keep_cols += [c for c in AOC_RRS_COLS if c in df_ins.columns]
    df_ins = df_ins[[c for c in keep_cols if c in df_ins.columns]].copy()
    return df_ins

def main():
    if not SUMMARY_CSV.exists():
        raise FileNotFoundError(f"Satellite summary not found: {SUMMARY_CSV}")
    if not INSITU_CSV.exists():
        raise FileNotFoundError(f"In-situ CSV not found: {INSITU_CSV}")

    df_sat_raw = pd.read_csv(SUMMARY_CSV)
    df_ins_raw = pd.read_csv(INSITU_CSV)

    # Prepare tables
    df_sat = parse_satellite_times(df_sat_raw)
    df_ins = load_insitu_aoc(df_ins_raw)

    # Split satellite rows by timestamp availability
    sat_valid   = df_sat.dropna(subset=["acq_datetime_utc"]).copy()
    sat_missing = df_sat[df_sat["acq_datetime_utc"].isna()].copy()

    # Sort for merge_asof
    sat_valid  = sat_valid.sort_values("acq_datetime_utc").reset_index(drop=True)
    ins_sorted = df_ins.sort_values("aoc_datetime").reset_index(drop=True)

    # Nearest-time join within ± HOURS_TOL
    if not sat_valid.empty and not ins_sorted.empty:
        right_cols = ["aoc_datetime","aoc_chl_a"] + [c for c in AOC_RRS_COLS if c in ins_sorted.columns]
        right = ins_sorted[right_cols].rename(columns={"aoc_datetime":"match_time"})

        merged = pd.merge_asof(
            sat_valid,
            right,
            left_on="acq_datetime_utc",
            right_on="match_time",
            direction="nearest",
            tolerance=pd.Timedelta(hours=HOURS_TOL)
        )

        merged["match_dt_hours"] = (
            (merged["match_time"] - merged["acq_datetime_utc"]).dt.total_seconds() / 3600.0
        )

        merged = merged.rename(columns={
            "match_time": "matched_aoc_datetime",
            "aoc_chl_a":  "matched_aoc_chl_a"
        })

        # Prefix any carried Rrs columns
        for c in AOC_RRS_COLS:
            if c in merged.columns:
                merged = merged.rename(columns={c: f"matched_{c}"})
    else:
        merged = sat_valid.copy()
        merged["matched_aoc_datetime"] = pd.NaT
        merged["matched_aoc_chl_a"]    = np.nan
        merged["match_dt_hours"]       = np.nan

    # Rows without satellite timestamps remain unmatched
    if not sat_missing.empty:
        sat_missing = sat_missing.copy()
        sat_missing["matched_aoc_datetime"] = pd.NaT
        sat_missing["matched_aoc_chl_a"]    = np.nan
        sat_missing["match_dt_hours"]       = np.nan

    # Stitch & sort
    out = pd.concat([merged, sat_missing], ignore_index=True)
    sort_keys = [k for k in ["acq_datetime_utc","filename","file"] if k in out.columns]
    if sort_keys:
        out = out.sort_values(sort_keys).reset_index(drop=True)

    # Write new file
    OUT_MATCHED_CSV.parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(OUT_MATCHED_CSV, index=False)

    # Report
    n_total   = len(out)
    n_matched = out["matched_aoc_chl_a"].notna().sum()
    n_no_time = sat_missing.shape[0]
    print(f"Matched AERONET-OC within ±{HOURS_TOL} h for {n_matched}/{n_total} satellite rows.")
    print(f"Satellite rows with missing timestamps (unmatchable): {n_no_time}")
    print(f"New file written: {OUT_MATCHED_CSV}")

if __name__ == "__main__":
    main()



Matched AERONET-OC within ±3 h for 42/58 satellite rows.
Satellite rows with missing timestamps (unmatchable): 0
New file written: Y:\Mingyue\Lucinda\Lucinda_l2_summary_2.csv
